## Libraries & Setup

### Setup

In [1]:
from functools import partial

import numpy as np
import pandas as pd
import torch.nn.parallel

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from utils.adaquant import optimize_layer_adaquant
from utils.load_dataset import load_dataset
from utils.utils_functions import val_loop, set_seeds
from utils.mobilenet_v2 import MobileNetV2
from utils.quantize import QConv2d, QLinear
from utils.trainer import Trainer

set_seeds()

## Experiments

For AdaQuant algorithm on already trained model - for 8, 16 and 32 bits.

In [5]:
bits=[8,16,32]

for bit in bits:
    acc = -1
    loss = -1
    best_prec1 = 0
    dtype = torch.float32


    #Load dataset
    cal_dataloader, train_dataloader, val_dataloader=load_dataset(
        './data/calibration',
        './data/calibration_labels.csv',
        './data/cifar-10/train',
        './data/cifar-10/trainLabels.csv'
        )
    
    #load model
    model = MobileNetV2(num_bits=bit, num_bits_weight=bit)
    model.load_state_dict(torch.load('models/mobilenet1.pt'))
    criterion = CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr=1e-2, momentum=0.5, weight_decay=0)
    trainer = Trainer(model, criterion, optimizer, device=torch.device("cpu"))

    cal_acc_before=val_loop(cal_dataloader, model, 'cpu')
    val_acc_before=val_loop(val_dataloader, model, 'cpu')
    print(f"Calibration accuracy before quantization for {bit} bits {cal_acc_before}.")
    print(f"Validation accuracy before quantization for {bit} bits {val_acc_before}.")

    #save accuracy
    with open(f"./output_adaquant/acc_before_{bit}.txt", 'w') as f:
        f.write("Calibration ")
        f.write(str(cal_acc_before))
        f.write("Validation ")
        f.write(str(val_acc_before))

    cached_qinput = {}

    #prepare for quantization
    def Qhook(name, module, input, output):
        if module not in cached_qinput:
            cached_qinput[module] = []
            # Meanwhile store data in the RAM.
            cached_qinput[module].append(input[0].detach().cpu())

    cached_input_output = {}


    def hook(name, module, input, output):
        if module not in cached_input_output:
            cached_input_output[module] = []
        # Meanwhile store data in the RAM.
        cached_input_output[module].append((input[0].detach().cpu(), output.detach().cpu()))
        
    for name, m in model.named_modules():
        if isinstance(m, QConv2d) or isinstance(m, QLinear):
            m.name = name

    handlers = []
    count = 0
    for name, m in model.named_modules():
        if isinstance(m, QConv2d) or isinstance(m, QLinear):
            m.quantize = False
            handlers.append(m.register_forward_hook(partial(hook, name)))
            count += 1

    trainer.validate(val_dataloader)

    for handler in handlers:
        handler.remove()

    for m in model.modules():
        if isinstance(m, QConv2d) or isinstance(m, QLinear):
            m.quantize = True

    mse_df = pd.DataFrame(
        index=np.arange(len(cached_input_output)),
        columns=["name", "bit", "shape", "mse_before", "mse_after", "acc"],
    )
    print_freq = 100
    evaluate = "./output_adaquant/evaluate"

    #run quantization
    for i, layer in enumerate(cached_input_output):
        if i > 0: 
            count = 0
            cached_qinput = {}
            for name, m in model.named_modules():
                if layer.name == name:
                    if count < 5:
                        handler = m.register_forward_hook(partial(Qhook, name))
                        count += 1
            # Store input/output for all quantizable layers
            trainer.validate(cal_dataloader)
            print("cashed quant Input%s" % layer.name)
            cached_input_output[layer][0] = (
                cached_qinput[layer][0],
                cached_input_output[layer][0][1],
            )
            handler.remove()
        print(
            "\nOptimize {}:{} for {} bit of shape {}".format(
                i, layer.name, layer.num_bits, layer.weight.shape
            )
        )

        mse_before, mse_after = optimize_layer_adaquant(layer, cached_input_output[layer])
        acc=val_loop(val_dataloader, model, 'cpu')

        print("\nMSE before optimization: {}".format(mse_before))
        print("MSE after optimization:  {}".format(mse_after))
        mse_df.loc[i, "name"] = layer.name
        mse_df.loc[i, "bit"] = layer.num_bits
        mse_df.loc[i, "shape"] = str(layer.weight.shape)
        mse_df.loc[i, "mse_before"] = mse_before
        mse_df.loc[i, "mse_after"] = mse_after
        mse_df.loc[i, "acc"] = acc

    #save the results and model
    mse_csv = evaluate + f"_mse_{bit}.csv"
    mse_df.to_csv(mse_csv)

    filename = evaluate + f"_adaquant_{bit}.pth"
    torch.save(model.state_dict(), filename)

    cal_acc_after=val_loop(cal_dataloader, model, 'cpu')
    val_acc_after=val_loop(val_dataloader, model, 'cpu')
    print(f"Calibration accuracy after quantization for {bit} bits {cal_acc_after}.")
    print(f"Validation accuracy after quantization for {bit} bits {val_acc_after}.")

    #save accuracy
    with open(f"./output_adaquant/acc_after_{bit}.txt", 'w') as f:
        f.write("Calibration ")
        f.write(str(cal_acc_after))
        f.write("Validation ")
        f.write(str(val_acc_after))

Calibration accuracy before quantization for 8 bits 43.94394394394394.
Train accuracy before quantization for 8 bits 44.928985797159434.

Optimize 0:features.0.0 for 8 bit of shape torch.Size([32, 3, 3, 3])


100%|██████████| 100/100 [00:01<00:00, 99.81it/s]



MSE before optimization: 0.2984926104545593
MSE after optimization:  0.03186314180493355
cashed quant Inputfeatures.1.conv.0.0

Optimize 1:features.1.conv.0.0 for 8 bit of shape torch.Size([32, 1, 3, 3])


100%|██████████| 100/100 [00:02<00:00, 49.49it/s]



MSE before optimization: 0.0023151671048253775
MSE after optimization:  0.0023015954066067934
cashed quant Inputfeatures.1.conv.1

Optimize 2:features.1.conv.1 for 8 bit of shape torch.Size([16, 32, 1, 1])


...


MSE before optimization: 1.0935533046722412
MSE after optimization:  0.529356062412262
cashed quant Inputfeatures.18.0

Optimize 51:features.18.0 for 32 bit of shape torch.Size([1280, 320, 1, 1])


100%|██████████| 100/100 [00:02<00:00, 48.80it/s]



MSE before optimization: 10.549941062927246
MSE after optimization:  10.134812355041504
cashed quant Inputclassifier.1

Optimize 52:classifier.1 for 32 bit of shape torch.Size([10, 1280])


100%|██████████| 100/100 [00:00<00:00, 185.94it/s]



MSE before optimization: 3355.57470703125
MSE after optimization:  3075.38037109375
Calibration accuracy after quantization for 32 bits 10.01001001001001.
Train accuracy after quantization for 32 bits 9.721944388877777.
